### Running our own Python-based web server: Flask

As you can see from the above "flow", in OAuth2 authentication, we will need to have our own web server, to receive the answers that come back from the API. Python can do that using Flask, a very lightweight webserver. Below you can find a small example of a webserver that will run on your machine at port 5000. 

**Please note that we have configured this webserver to shutdown after a responding to a single call (we added the "stop\_server()" function call in call responses). In general, we do not do that, and we leave the server running.**

Once you run the call, visit http://yourIP:5000/ or http://yourIP:5000/testNYU to see the results. The server will stop immediately after, and the cell will stop being in "Busy" status.

In [1]:
# In this cell, we configure our example web server. 
# We will show in the next cell how to configure this
# web server to handle the OAuth2 calls.

# Flask is a webserver
from flask import Flask, request

# We add a global variable that will be used to count the visitors to a specific URL
visitor_counter = 0
# Initialize the embedded web server
webserver = Flask("MyFirstWebServer")

import datetime
import time
# Go to http://<your IP>:5000/ to see the response
@webserver.route('/')
def hello_world():
    # stop_server()
    return 'Hello Panos, it is now ' + str(datetime.date.today()) 

# Go to http://<your IP>:5000/testNYU to see the different message
@webserver.route('/testNYU')
def hello_nyu():
    # stop_server()
    global visitor_counter
    visitor_counter += 1
    return 'Hello! You are visitor #{i}'.format(i=visitor_counter)

def start_server():
    global visitor_counter
    visitor_counter = 0
    webserver.run(host='0.0.0.0', port=5000)
    return
    
def stop_server():
    shutdown_after_request = request.environ.get('werkzeug.server.shutdown')
    shutdown_after_request()
    return

start_server()

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)


#### Exercise

Consult the notes in 1/D, and figure out how to run the server in the background.